# Common Time Lightcurves

In [1]:
import os
os.chdir('../../scripts')
import pandas as pd
import matplotlib.pyplot as plt
from post_processing import get_output
from analysis_tools_cython import *
import seaborn as sns
plt.rcParams['figure.figsize'] = [12,7]

In [2]:
data = get_output('../eleanor-lite-combined-v2.txt')

ValueError: Length mismatch: Expected axis has 1 elements, new values have 25 elements

In [ ]:
data